# Export EIA 860

In [1]:
import pandas as pd

from oge.load_data import load_pudl_table

Load data

In [ ]:
gens_eia860 = load_pudl_table("core_eia860__scd_generators")
plants_eia860 = load_pudl_table("core_eia860__scd_plants")
plants_entity = load_pudl_table("core_eia__entity_plants")
generators_entity = load_pudl_table("core_eia__entity_generators")

Merge data

In [3]:
eia_gens = gens_eia860.merge(
    plants_eia860,
    how="left",
    on=["report_date", "plant_id_eia"],
    suffixes=(None, "_remove"),
    validate="m:1",
)

eia_gens = eia_gens.merge(
    generators_entity,
    how="left",
    on=["plant_id_eia", "generator_id"],
    suffixes=(None, "_remove"),
    validate="m:1",
)

eia_gens = eia_gens.merge(
    plants_entity,
    how="left",
    on=["plant_id_eia"],
    suffixes=(None, "_remove"),
    validate="m:1",
)

Keep only the most recent entry for each generator

In [4]:
eia_gens = eia_gens.sort_values(
    by=["plant_id_eia", "generator_id", "report_date"], ascending=True
)
eia_gens = eia_gens.drop_duplicates(
    subset=["plant_id_eia", "generator_id"], keep="last"
)

Remove duplicate columns

In [5]:
eia_gens = eia_gens[[col for col in eia_gens.columns if "_remove" not in col]]
eia_gens = eia_gens[
    [
        "plant_id_eia",
        "plant_name_eia",
        "generator_id",
        "report_date",
        "rto_iso_lmp_node_id",
        "transmission_distribution_owner_name",
        "transmission_distribution_owner_state",
        "balancing_authority_code_eia",
        "balancing_authority_name_eia",
        "iso_rto_code",
        "nerc_region",
        "latitude",
        "longitude",
        "street_address",
        "city",
        "county",
        "state",
        "zip_code",
        "capacity_mw",
        "grid_voltage_1_kv",
        "prime_mover_code",
        "energy_source_code_1",
        "energy_source_code_2",
        "multiple_fuels",
        "cofire_fuels",
        "switch_oil_gas",
        "energy_storage",
        "generator_operating_date",
        "current_planned_generator_operating_date",
        "operational_status",
        "generator_retirement_date",
    ]
]

Keep only states in MISO footprint

In [6]:
states_to_remove = [
    "AK",
    "HI",
    "WA",
    "OR",
    "CA",
    "ID",
    "NV",
    "AZ",
    "UT",
    "WY",
    "CO",
    "ME",
    "NH",
    "RI",
]

# to filter this a bit, get rid of generators in WECC, Texas, Alaska, and Hawaii
eia_gens = eia_gens[~eia_gens["state"].isin(["UNK", "TRE", "WECC"])]

Export file

In [7]:
eia_gens.to_csv("~/Desktop/eia860_export.csv", index=False)